# 1 map

map()函数用于将一个函数应用到一个或多个可迭代对象（列表、字符串、元组、series）的每个元素，并返回一个新的迭代器

## 1.1 基本使用场景

### 1.1.1 基本语法

map(function, iterable,...)

- 应用的函数(函数名称)
- iterable，处理的可迭代对象
- 如果有多个可迭代对象，map函数从每个可迭代对象依次取一个元素作为参数

返回值：一个map对象，迭代器，使用list、tuple等转化为具体结构

### 1.1.2 基本使用示例

#### 1.1.2.1 使用内置函数

In [1]:
import struct

strings = ['apple','banana','bear','purple']
res = map(len,strings)
list(res)

[5, 6, 4, 6]

#### 1.1.2.2 使用自定义函数

In [3]:
def len_plus_two(x):
    return len(x)+2

res = map(len_plus_two,strings)
list(res)

[7, 8, 6, 8]

#### 1.1.2.3 使用匿名函数

In [4]:
res = map(lambda x:len(x)+3,strings)
list(res)

[8, 9, 7, 9]

#### 1.1.2.4 传入多个可迭代对象

In [5]:
num1 = [1,2,3]
num2 = [4,5,6]
res = map(lambda x,y:x*y,num1,num2)
list(res)

[4, 10, 18]

## 1.2 map应用于series

在pandas中提供了用于series的map函数，用于对series中的每个元素进行操作

### 1.2.1 基本语法

Series.map(arg)
- arg:可以是函数(具体写函数名)，也可以是字典（用于值的映射和替换）
- 不会改变原来的行，需要重新赋值

> 列表表达式要求用函数的调用而不是名称

> s = pd.Series({'a':[1, 2, 3, 4, 5, 6]})和s = pd.Series([1, 2, 3, 4, 5, 6])的差别

In [9]:
import pandas as pd
s = pd.Series({'a':[1, 2, 3, 4, 5, 6]})
s

a    [1, 2, 3, 4, 5, 6]
dtype: object

In [11]:
s = pd.Series(['apple','banana','bear','purple'])
s

0     apple
1    banana
2      bear
3    purple
dtype: object

### 1.2.2 Series中的map使用示例

#### 1.2.2.1 arg为函数，进行函数映射

In [17]:
s = pd.Series(['apple','banana','pear','grace'])
# 全转为大写
s.map(str.upper)

0     APPLE
1    BANANA
2      PEAR
3     GRACE
dtype: object

In [13]:
def squared(x):
    return x*2
s.map(squared)

0      appleapple
1    bananabanana
2        bearbear
3    purplepurple
dtype: object

In [15]:
s.map(lambda x:x*3)

0       appleappleapple
1    bananabananabanana
2          bearbearbear
3    purplepurplepurple
dtype: object

#### 1.2.2.2 arg字典映射

In [18]:
# arg作为字典进行替换
mapping = {'apple':'red','banana':'yellow','pear':'yellow','grace':'purple'}
s.map(mapping)

0       red
1    yellow
2    yellow
3    purple
dtype: object

# 2 apply函数

用于对DataFrame或Series中的元素逐个按行、列进行操作，实现复杂的数据处理

## 2.1 基本语法

df.apply(function,axis)

- function:要应用的函数
- axis:控制函数的应用方式
    - 0：对每一列应用函数，默认值
    - 1：对每一行应用函数

## 2.2 apply函数使用场景

### 2.2.1 应用在DataFrame（多行或多列操作）

#### 2.2.1.1 应用在DataFrame的单列，axis=0，并添加新列


In [19]:
import numpy as np

In [23]:
np.random.seed(1)

data = np.random.randint(1,100,size=(10,5))
data = pd.DataFrame(data=data,columns=['a','b','c','d','e'])
data

,a,b,c,d,e
0,38,13,73,10,76
1,6,80,65,17,2
2,77,72,7,26,51
3,21,19,85,12,29
4,30,15,51,69,88
5,88,95,97,87,14
6,10,8,64,62,23
7,58,2,1,61,82
8,9,89,14,48,73
9,31,72,4,71,22


In [24]:
data.apply(np.sum,axis=0)

a    368
b    465
c    461
d    463
e    460
dtype: int64

In [25]:
data.apply(np.sum,axis=1)

0    210
1    170
2    233
3    166
4    253
5    381
6    167
7    204
8    233
9    200
dtype: int64

In [28]:
# 自定义函数，根据数字大小分段
def sperate(x):
    if x <= 30:
        return "p1"
    elif x <= 60:
        return "p2"
    else:
        return "p3"

data['e'] = data['a'].apply(sperate)
data

,a,b,c,d,e
0,38,13,73,10,p2
1,6,80,65,17,p1
2,77,72,7,26,p3
3,21,19,85,12,p1
4,30,15,51,69,p1
5,88,95,97,87,p3
6,10,8,64,62,p1
7,58,2,1,61,p2
8,9,89,14,48,p1
9,31,72,4,71,p2


#### 2.2.1.2 应用在DataFrame的多列，axis=1，返回单个值

In [29]:
data['total'] = data[['a','b']].apply(np.sum,axis=1)
data

,a,b,c,d,e,total
0,38,13,73,10,p2,51
1,6,80,65,17,p1,86
2,77,72,7,26,p3,149
3,21,19,85,12,p1,40
4,30,15,51,69,p1,45
5,88,95,97,87,p3,183
6,10,8,64,62,p1,18
7,58,2,1,61,p2,60
8,9,89,14,48,p1,98
9,31,72,4,71,p2,103


#### 2.2.1.2 应用在DataFrame的多列，axis=1，返回多个值

In [31]:
def calculate(row):
    sum_v = row['a'] + row['b']
    diff_v = row['a'] - row['b']
    return pd.Series([sum_v,diff_v],index=['sum','diff'])
data[['sum','Diff']] = data[['a','b']].apply(calculate,axis=1)
data

,a,b,c,d,e,total,sum,Diff
0,38,13,73,10,p2,51,51,25
1,6,80,65,17,p1,86,86,-74
2,77,72,7,26,p3,149,149,5
3,21,19,85,12,p1,40,40,2
4,30,15,51,69,p1,45,45,15
5,88,95,97,87,p3,183,183,-7
6,10,8,64,62,p1,18,18,2
7,58,2,1,61,p2,60,60,56
8,9,89,14,48,p1,98,98,-80
9,31,72,4,71,p2,103,103,-41


# 3 parallel_apply

当数据很大时，apply效率低下，使用parallel_apply并行加快效率,需要安装pandarallel包

用法：

1. 初始化pandarallel:pandarallel.initialize()
2. 调用parallel_apply:语法和apply完全一致
3. windows上有问题

其他并行库
https://blog.csdn.net/weixin_35757704/article/details/121991122

In [ ]:
!pip install pandarallel

In [1]:
import time
time.time()

1761579341.0026178

In [2]:
import pandas as pd
from pandarallel import pandarallel
# 创建一个大DF
df = pd.DataFrame({'a':range(1,1000001),'b':range(1,1000001),'c':range(1,1000001)})
df

,a,b,c
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
...,...,...,...
999995,999996,999996,999996
999996,999997,999997,999997
999997,999998,999998,999998
999998,999999,999999,999999


In [11]:
def complex_function(x):
    time.sleep(0.0000001)# 模拟一个耗时操作
    return x*2

In [ ]:
# 初始化pandarallel
pandarallel.initialize()
# 记录开始时间
start_time = time.time()
# 使用parallel_apply并行处理
df['d'] = df['a'].parallel_apply(complex_function)
# 记录结束时间
end_time = time.time()
# 计算耗时
print(f'耗时{end_time - start_time}')

# 4 性能比较

apply,parallel_apply，iterrows，itertuples

| 方法             | 优点              | 缺点                        | 性能（大数据集） | 推荐场景                     |
|----------------|-----------------|---------------------------|----------|--------------------------|
| apply          | 灵活，支持任意函数       | 性能较差，不支持并行，较慢             | 中等       | 小到中型数据集，操作复杂，且不需要并行化     |
| parallel_apply | 支持并行，加速计算       | 复杂，需要额外库支持，可能增加开销         | 快        | 大型数据集，且需要并行化操作           |
| iterrows       | 简单易用，逐行处理       | 性能差，返回Series，内存开销大        | 较慢       | 小型数据集需要返回行索引的情况          |
| itertuples     | 性能最好，返回元组，内存效率高 | 不如apply灵活，不能直接修改DataFrame | 非常快      | 需要高性能的逐行处理，无需修改DataFrame |